<div dir=rtl align=center>

<img src='https://upload.wikimedia.org/wikipedia/fa/thumb/a/a9/Sharif_logo.svg/626px-Sharif_logo.svg.png?20110526112825' alt="SUT logo" width=200 height=200 align=center  >
<br>
<font face="B Yekan">
<font color=0F5298 size=7>
یادگیری ماشین<br>
<font color=2565AE size=5>
دانشکده مهندسی صنایع<br>
<font color=2565AE size=4>
دکتر مهدی شریف زاده <br>
<font  size=4>
 امیرحسین محمودی <br>
بهار 1402<br>

<font color=3C99D size=5>
ماشین بردار پشتیبان (SVM)
<br>
    
    
    
____


Support Vector Machines

SVM is a powerful and versetile ML model that is capable of performing Classification, Regression, & even outlier detection. 

SVMs are particularly suited for complex small-to-medium sized datasets.

## Linear SVM Classification

The fundamental idea behind SVMs is better explained with the following picture:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/SVM_example.png" /></div>

We can see that the two classes can be separated easily by a straight line (linearly separable). The left plot shows the decision boundaries of three possible linear classifiers. The dashed line model is so bad that it doesn't even separate the two groups linearly. The other two models work perfectly on the plotted training set but their boundaries are so close to the training data points that they'll probably not perform well on unseen data.

In constrast, the model on the right not only separate the training data linearly, it also stays as far as possible from both classes data points. Thus, it will likely perform well on unseen data.

We can think of an SVM as fitting the widest possible street (represented by the dashed lines) between the classes. This is called **Large Margin Classification**.

We should notice that adding more training points off the street **won't effect the decision boundary at all**. It's fully determined by the data points located at the edge of the street. These instances are called **support vectors**.

SVMs are also sensitive to feature scales.

### Soft Margin Classification

If we restrict all training instances to be off the SVM street, this is called Hard Margin Classification, Hard Margin Classification is greatly effected by the presence of outliers. The following are two example of how outliers can mess-up hard margin classifiers:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/Hard_Margin_Classifier.png" /></div>

To fix the issue, we try to balance finding a wide street with limiting the number of violations. This is called **Soft Margin Classification**.

This can be controlled in scikit-learn by the `C` hyper-parameter:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/Soft_to_Hard_Street.png" /></div>

By increasing `C`, We're increasing the sensitivity of the model to minimize margin violations within the training set. Meaning, If we're overfitting, we try to reduce the value of the `C` hyper-parameter.

Let's use scikit-learn's SVMs:

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [2]:
iris = datasets.load_iris()

In [3]:
iris['data'].shape

(150, 4)

In [4]:
X = iris['data'][:, [2,3]]  # Petal Length, Petal Width
y = (iris['target'] == 2).astype(np.float64)  # Iris Virginica
X.shape, y.shape

((150, 2), (150,))

In [5]:
svm_clf = Pipeline([
    ('Scaler', StandardScaler()),
    ('Linear_svc', LinearSVC(C=1, loss='hinge'))
])

In [6]:
svm_clf.fit(X, y)

Pipeline(steps=[('Scaler', StandardScaler()),
                ('Linear_svc', LinearSVC(C=1, loss='hinge'))])

In [7]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

Unlike Logistic Regression Models (with their sigmoid functions), SVMs do not output probabilities for each class.

## Hinge Loss Function
Hinge Loss
The hinge loss is a specific type of cost function that incorporates a margin or distance from the classification boundary into the cost calculation. Even if new observations are classified correctly, they can incur a penalty if the margin from the decision boundary is not large enough. The hinge loss increases linearly.

The hinge loss is mostly associated with soft-margin support vector machines.

<div style="text-align:center;"><img style="width:33%;" src="static/imgs/hingel-loss.png" /></div>
If you are familiar with the construction of hyperplanes and their margins in support vector machines, you probably know that margins are often defined as having a distance equal to 1 from the data-separating-hyperplane. We want data points to not only fall on the correct side of the hyperplane but also to be located beyond the margin.

<div style="text-align:center;"><img style="width:33%;" src="static/imgs/svm-margin.png" /></div>
Support vector machines address a classification problem where observations either have an outcome of +1 or -1. The support vector machine produces a real-valued output that is negative or positive depending on which side of the decision boundary it falls. Only if an observation is classified correctly and the distance from the plane is larger than the margin will it incur no penalty. The distance from the hyperplane can be regarded as a measure of confidence. The further an observation lies from the plane, the more confident it is in the classification.
For example, if an observation was associated with an actual outcome of +1, and the SVM produced an output of 1.5, the loss would equal 0.

Contrary to methods like linear regression, where we try to find a line that minimizes the distance from the data points, an SVM tries to maximize the distance. If you are interested, check out my post on constructing regression lines. Comparing the two approaches nicely illustrates the difference between the nature of regression and classification problems.


<div style="text-align:center;"><img style="width:33%;" src="static/imgs/hinge-0-1536x818.png" /></div>
An observation that is located directly on the boundary would incur a loss of 1 regardless of whether the real outcome was +1 or -1.

svm with an observation on the decision boundary and its hinge loss

<div style="text-align:center;"><img style="width:33%;" src="static/imgs/Screenshot-2021-08-20-at-05.46.39-1536x835.png" /></div>
Observations that fall on the correct side of the decision boundary (hyperplane) but are within the margin incur a cost between 0 and 1.

svm with an observation within the margin and its hinge loss

<div style="text-align:center;"><img style="width:33%;" src="static/imgs/Screenshot-2021-08-20-at-05.59.38-1536x772.png" /></div>
All observations that end up on the wrong side of the hyperplane will incur a loss that is greater than 1 and increases linearly. If the actual outcome was 1 and the classifier predicted 0.5, the corresponding loss would be 0.5 even though the classification is correct.

Now that we have a strong intuitive understanding of the hinge loss, understanding the math will be a breeze.

l(y)=max(0,1−t⋅y)

## NonLinear SVM Classification

Many datasets are not even close to being lienarly separable. One approach to handling non-linear modeling is to add more features, such as polynomial features. In some cases this can result in linearly separable datasets.

The following is an example of an original non-linearly separable dataset with only one feature $x_{1}$ (on the left), and an augmented linearly seprable dataset with an added feature $x_{2}=x_{1}^{2}$: 

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/nonlinear_to_linear.png" /></div>

Let's implement this idea using scikit-learn:

In [8]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

In [9]:
X, y = make_moons(n_samples=100, noise=0.15)

In [10]:
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

In [11]:
polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

In [12]:
polynomial_svm_clf.score(X, y)

1.0

The following represents the decision boundaries of the model, because we added polynomial degrees, projected boundaries are now non-linear:

<div style="text-align:center;"><img style="width:50%;" src="static/imgs/polynomial_svms.png" /></div>

### Polynomial Kernels

At a low polynomial degrees, adding features cannot deal with complex datasets, and for high polynomial degrees, we endup adding a lot of features, resulting in a very complex & slow model.

Fortunately, when using SVMs we can apply an almost miraculous mathematical technique called the **kernel trick**. The kernel trick makes it possible to have the same result as if we added many polynomial features without actually adding them.

Let's test it on the moon dataset:

In [13]:
from sklearn.svm import SVC

In [14]:
poly_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5))
])

In [15]:
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

This model trains an SVM classifier using a kernel of third degree features.

If our model is overfitting, we might want to decrease the polynomial degree, and If it's underfitting, it might be a good idea to increase the degree. `coef0` controls how much the model is influenced by high-degree polynomials vs. low degree polynomials.

The following figure shows the previously trained model (on the left) vs. a more complex model of kernel degree 10:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/kernel_trick.png" /></div>

### Similarity Features

Another technique to tackle non-linear problems is to add features computed using a **similarity function**, which measures how much each instance resembles a particular landmark.

For example, let's take the 1D dataset discussed earlier & add two landmarks to it at $x_{1}=-2$ and $x_{1}=1$, as showcased in the left plot of:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/similarity_measures.png" /></div>

We defined the similarity function to be the **Gaussian Radial Basis Function (RBF)** with $\gamma = 0.3$. $\phi$ is a bell shaped function varying from 0 ($x$ is very far from $l$) to 1 ($x=l$). After defining a similarity function, the new features are the **distances** of each training instance from the landmarks.

As we can see from the plot on the right, the instances become lienarly separable using only distance features.

But how do we select the landmarks? The simplest approach is to create a landmark at each and every point of the training data. Doing that will increase the number of dimensions and will yield a better chance of finding a linear separator. The downside is that we will create additional `m` number of features (as rows), which may lead in performance issues.

### Gaussian RBF Kernel

Just like the polynomial features method, the similarity features method can be useful in many ML algorithms, the problem is that with very large datasets, we'll endup with a very big feature space, but once again we have the Kernel trick to make it look as if we added the additional features.

Let's do it with `sklearn`:

In [16]:
from sklearn.svm import SVC

In [17]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

In [18]:
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

Let's take a look at the predictions space with the training set instances (bottom left is the trained model above), others correspond to different hyper-parameter configurations:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/training_rbfs.png" /></div>

Increasing $\gamma$ makes the bell-shaped curve narrower, the decision boundary ends up being more irregular, wiggling around individual instances. So $\gamma$ acts as a regularization hyper-parameter.
- Increasing gamma increases model sensitivity (may lead to overfitting).
- decreasing gamma increases model bias (may lead to underfitting)

We should always try the linear kernel first, if the training set is not too large, we should also try the gaussian RBF kernel.

### Computational Complexity

`LinearSVC` doesn't support the kernel trick and scales with time complexity of $\mathcal{O}(m \times n)$. Its algorithm takes longer if we ask for higher precision, precision is controlled by the tollerance hyper-parameter $\epsilon$.

`SVC` is based on `libsvm` that supports the kernel trick with complexity between $\mathcal{O}(m^{2} \times n)$ & $\mathcal{O}(m^{3} \times n)$. This means that it gets dreadfuly slow when the training instances count gets big. This algorithm is good for small to medium sized datasets and scales well with the number of features.

## SVM Regression

SVMs also support linear and nonlinear regression, but to move from classification to regression we have to reverse the objective. Instead of trying to fit the largest possible street between two classes while limiting margin violations, SVM regression tries to fit **as many instance as possible** on the street while limiting margin violations. The width of the street is controlled by the hyper-parameter $\epsilon$, following is an example:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/SVM_regression.png" /></div>

Adding more instances into the margin doesn't effect the model's predictions; thus the model is said to be $\epsilon$-insensitive.

Let's implement it using `sklearn`'s `SVR` (after scaling & centering the data):

In [19]:
from sklearn.svm import LinearSVR

In [20]:
svm_reg = LinearSVR(epsilon=1.5)

In [21]:
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

To tackle linear regression tasks, we can use a kernelized SVM model.

Let's do it using the polynomial kernel:

In [22]:
from sklearn.svm import SVR

In [23]:
svm_poly_reg = SVR(kernel='poly', degree=2, C=100, epsilon=0.1, gamma='auto')

In [24]:
svm_poly_reg.fit(X, y)

SVR(C=100, degree=2, gamma='auto', kernel='poly')

`LinearSVR` scales linearly with the size of the training set, while `SVR` is much slower (just like `LinearSVC` & `SVC`).

## Under the Hood

This section explains how `SVMs` make predictions and how their training algorithms work, starting with Linear SVM classifiers.

### Decision Function & Predictions

The linear SVM classifier model predicts the class of a new instance by simply computing $w^{T}x+b = w_1x_1+w_2x_2+\dots+w_nx_n+b$, If it's positive, then $x$ is labeled $1$, otherwise $x$ is labeled $0$:

$$\hat{y}=\begin{cases}
1,  & \text{if $w^{T}x+b \ge 0$} \\
0, & \text{if $w^{T}x+b < 0$}
\end{cases}$$

The following showcases the training data points and the activation $w^{T}x+b$:

<div style="text-align:center;"><img style="width:66%;" src="static/imgs/SVM_space.png" /></div>

Training an SVM Classifier is finding a $(w,b)$ that makes the margin as wide as possible while avoiding margin violations (hard margin) or limiting them (soft margin).

### Training Objective

We know that the slope of the decision function is $||w||$. Dividing $||w||$ by $2$ will multiply the margin by $2$. So we want to minimize $||w||$ to get a large margin. 

If we also want to avoid any margin violations then:
- We want the decision function to be greater than 1 for all positive training instances.
- And lower than -1 for all negative training instances.

If we define $t^{(i)}=-1$ for negative instances (if $y^{(i)}=0$) and $t^{(i)}=1$ for positive instances (if $y^{(i)}=1$). Then we can express this constraint as $t^{(i)}(w^{T}x^{(i)}+b)\ge 1$. 

We can therefore express the hard margin linear SVM classifier objective as the following constrained optimization problem:

$$\begin{array}{ll}
\text{minimize}_{(w,b)}  & \frac{1}{2}w^{T}w \\
\text{subject to}& t^{(i)}(w^{T}x^{(i)} + b) \ge 1; \forall i \in \{1,2,\dots,m\}
\end{array}$$

To get the soft margin objective, we need to introduce a slack variable $\zeta^{(i)} \ge 0$ for each instance. $\zeta^{(i)}$ measures how much the i-th instance is allowed to violate the margin.

We now have two conflicting objectives, Make the slack variables as small as possible to reduce the margin violations, and make $\frac{1}{2}w^{T}w$ as small as possible to increase the margin. 

This is where the $C$ hyper-parameter comes in: It allows to define the tradeoff between these two objectives. This gives us the following contrained optimization problem for Softmargin Linear SVM Classifier Objective:
    
$$\begin{array}{ll}
\text{minimize}_{(w,b,\zeta)}  & \frac{1}{2}w^{T}w + C\sum_{i=1}^{m}\zeta^{(i)} \\
\text{subject to}& t^{(i)}(w^{T}x^{(i)} + b) \ge 1 - \zeta^{(i)}; \zeta^{(i)} \ge 0 \; \forall i \in \{1,2,\dots,m\}
\end{array}$$

### Quadratic Programming

The hard/soft margin problems are both quadratic optimization problems with linear constraints. Such problems are known as *Quadratic Programming* (QP) problems.

One way to train a hard margin linear SVM classifier is to use an off-the-shelf QP Solver and pass it our parameters. We can also use a Quadratic Solver to train for the soft margin linear SVM classifier.

To use the kernel trick though, we're going to take a look at a different optimization problem:

### The Dual Problem

Given a constrained optimization problem (Primal problem) it is possible to express a different but closely related problem (dual problem).

The solution to the dual problem typically gives a lower bound to the solution of the primal problem, but under some conditions it may have the same solution as the primal problem. Luckily, the SVM problem happens to meet these conditions.

Here is the dual problem formulation for the SVM Linear classification optimization problem:

$$\begin{array}{ll}
\text{minimize}_{(\alpha)}  & \frac{1}{2}\sum_{i=1}^{m}\sum_{j=1}^{m}\alpha^{(i)}\alpha^{(j)}t^{(i)}t^{(j)}{x^{(i)}}^{T}x^{(j)} - \sum_{i=1}^{m}\alpha^{(i)} \\
\text{subject to}& \alpha^{(i)} \ge 0; \forall i \in \{1,2,\dots,m\}
\end{array}$$

Once we find the solution $\hat{\alpha}$ to the previous problem, we use the following to calculate its corresponding $\hat{w}$ abd $\hat{b}$:

$$\hat{w}=\sum_{i=1}^{m}\hat{w}^{(i)}t^{(i)}x^{(i)} \\ \hat{b} = \frac{1}{n_s} \sum_{i=1}^{m}(t^{(i)} - \hat{w}^{T}x^{(i)})$$

The dual problem is faster to solve when the number of instances is less then the number of features. More importantly, the dual problem makes the kernel trick possible while the primal problem does not, so what is the kernel trick anyway?

### Kernelized SVMs

If we apply the transformation $\phi$ to all training instances, then the dual problem will contain the dot product $\phi(x^{(i)})^{T}\phi(x^{(j)})$. But if $\phi$ is the second degree polynomial transformation then we can replace $\phi(x^{(i)})^{T}\phi(x^{(j)})$ with $({x^{(i)}}^{T}x^{(j)})^{2}$. Meaning, **We don't need to transform the instances at all** (this trick makes for a much simpler computation).

The function $K(a,b)=(a^{T}b)^{2}$ is a 2nd degree polynomial kernel. In ML, a kernel is a function capable of computing $\phi(a)^{T}\phi(b)$ based only on the original vectors $a$ and $b$ without having to compute the original transformation $\phi$

In the case of the gaussian RBF kernel: $K(a,b)=exp(-\gamma||a-b||^{2})$, It can be shown that $\phi$ maps each instance to an infinite dimensional space, so it's a good thing we don't actually need to perform the mapping.

### Online SVMs

We should remember that online learning means learning incrementally, as new instances arrive. For linear SVMs classifiers, one way to implement online learning is through gradient descent to minimize the cost function derived from the primal objective. Unfortunately, GD converges much slower than the QP methods.

For large-scale non-linear problems, we may want to consider using neural networks instead.

---